In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm

Location="titanic3.xls"
data = pd.read_excel(Location)

In [2]:
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [2]:
newdata=data
def number_relatives(x,y):
    return x+y
def sex_to_number(x):
    if x == 'female':
        return 1
    if x == 'male':
        return 0
newdata['sexnum']=data['sex'].apply(sex_to_number)
newdata['onboard.relatives']=number_relatives(data['sibsp'],data['parch'])
newdata.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,sexnum,onboard.relatives
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO",1,0
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON",0,3
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",1,3
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON",0,3
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",1,3


In [3]:
# getting an overview of our data
print("Our data has {0} rows and {1} columns".format(data.shape[0], data.shape[1]))
# checking for missing values
print("Are there missing values? {}".format(data.isnull().any().any()))
data.describe()

Our data has 1309 rows and 16 columns
Are there missing values? True


,pclass,survived,age,sibsp,parch,fare,body,sexnum,onboard.relatives
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,121.000000,1309.000000,1309.000000
mean,2.294882,0.381971,29.881135,0.498854,0.385027,33.295479,160.809917,0.355997,0.883881
std,0.837836,0.486055,14.413500,1.041658,0.865560,51.758668,97.696922,0.478997,1.583639
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800,72.000000,0.000000,0.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,155.000000,0.000000,0.000000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000,256.000000,1.000000,1.000000
max,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200,328.000000,1.000000,10.000000


I like to take a general look at the data before I really get started. 
The first thing I notice is that there 263 missing values in the age field.
The second thing I notice is that very few bodies were recovered.

In [4]:
data.corr()

,pclass,survived,age,sibsp,parch,fare,body,sexnum,onboard.relatives
pclass,1.000000,-0.312469,-0.408106,0.060832,0.018322,-0.558629,-0.034642,-0.124617,0.050027
survived,-0.312469,1.000000,-0.055513,-0.027825,0.082660,0.244265,NaN,0.528693,0.026876
age,-0.408106,-0.055513,1.000000,-0.243699,-0.150917,0.178739,0.058809,-0.063646,-0.240230
sibsp,0.060832,-0.027825,-0.243699,1.000000,0.373587,0.160238,-0.099961,0.109609,0.861952
parch,0.018322,0.082660,-0.150917,0.373587,1.000000,0.221539,0.051099,0.213125,0.792296
fare,-0.558629,0.244265,0.178739,0.160238,0.221539,1.000000,-0.043110,0.185523,0.226492
body,-0.034642,NaN,0.058809,-0.099961,0.051099,-0.043110,1.000000,0.015903,-0.033628
sexnum,-0.124617,0.528693,-0.063646,0.109609,0.213125,0.185523,0.015903,1.000000,0.188583
onboard.relatives,0.050027,0.026876,-0.240230,0.861952,0.792296,0.226492,-0.033628,0.188583,1.000000


The values with the strongest correlation with Survival are passenger class, sex, age, and fare.

In [5]:
result = sm.ols(formula='survived ~ pclass + sexnum + age + fare', data=data).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               survived   R-squared:                       0.368
Model:                            OLS   Adj. R-squared:                  0.366
Method:                 Least Squares   F-statistic:                     151.6
Date:                Fri, 12 Oct 2018   Prob (F-statistic):          3.54e-102
Time:                        20:24:57   Log-Likelihood:                -500.69
No. Observations:                1045   AIC:                             1011.
Df Residuals:                    1040   BIC:                             1036.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7717      0.066     11.693      0.000       0.642       0.901
pclass        -0.1786      0.019     -9.415      0.000      -0.216      -0.141
sexnum         0.4902      0.026     19.024      0.000       0.440       0.541
age           -0.0052      0.001     -5.561      0.000      -0.007      -0.003
fare           0.0001      0.000      0.431      0.667      -0.000       0.001
==============================================================================
Omnibus:                       36.829   Durbin-Watson:                   1.891
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               40.210
Skew:                           0.480   Prob(JB):                     1.86e-09
Kurtosis:                       2.965   Cond. No.                         394.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The P value for fare is too high so I will try the regression without it.

In [6]:
result = sm.ols(formula='survived ~ pclass + sexnum + age', data=data).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               survived   R-squared:                       0.369
Model:                            OLS   Adj. R-squared:                  0.367
Method:                 Least Squares   F-statistic:                     202.8
Date:                Fri, 12 Oct 2018   Prob (F-statistic):          1.44e-103
Time:                        20:25:10   Log-Likelihood:                -500.78
No. Observations:                1046   AIC:                             1010.
Df Residuals:                    1042   BIC:                             1029.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7848      0.057     13.688      0.000       0.672       0.897
pclass        -0.1828      0.016    -11.395      0.000      -0.214      -0.151
sexnum         0.4915      0.026     19.236      0.000       0.441       0.542
age           -0.0052      0.001     -5.600      0.000      -0.007      -0.003
==============================================================================
Omnibus:                       36.642   Durbin-Watson:                   1.894
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               39.979
Skew:                           0.479   Prob(JB):                     2.08e-09
Kurtosis:                       2.968   Cond. No.                         164.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The adjusted r-square values is very, very slightly better and all of the P values are 0.

In [7]:
pd.pivot_table(data, index= 'home.dest', values= "survived", aggfunc='count', fill_value= 0,
              margins = True, margins_name= 'Total Survivors')

,survived
home.dest,
"?Havana, Cuba",1
"Aberdeen / Portland, OR",2
"Albany, NY",1
"Altdorf, Switzerland",2
"Amenia, ND",2
"Antwerp, Belgium / Stanton, OH",1
Argentina,1
"Asarum, Sweden Brooklyn, NY",1
"Ascot, Berkshire / Rochester, NY",1


In [17]:
pd.pivot_table(data, index= ['pclass', 'sex'], values= "survived", aggfunc='sum', fill_value= 0,
              margins = True, margins_name= 'Total Survivors')

survived
pclass          sex             
1               female       139
                male          61
2               female        94
                male          25
3               female       106
                male          75
Total Survivors              500

Just looking at the number of survivors by passenger class and sex.

In [13]:
pd.pivot_table(data, index= ['pclass', 'survived'], values= "age", aggfunc= [np.mean, np.median, min, max, np.std], fill_value= 0,
              margins = True, margins_name= 'Total Survivors')

mean median      min max        std
                                age    age      age age        age
pclass          survived                                          
1               0         43.247573     45   2.0000  71  14.227299
                1         36.833794     36   0.9167  80  14.249562
2               0         33.167808     30  14.0000  70  12.235234
                1         24.858696     25   0.6667  62  13.959568
3               0         25.974549     25   0.3333  74  11.880771
                1         21.545166     22   0.1667  63  11.607635
Total Survivors           29.881135     28   0.1667  80  14.406608

In [18]:
def remove_outliers(values):
    mid_quantiles = values.quantile([.25, .75])
    return np.mean(mid_quantiles)
pd.pivot_table(data, index= ['sex', 'survived'], values= "age", aggfunc= [np.mean, remove_outliers, np.median, min, max, np.std], fill_value= 0,
              margins = True, margins_name= 'Total Survivors')

mean remove_outliers median     min max  \
                                age             age    age     age age   
sex             survived                                                 
female          0         25.255208          24.625   24.5  1.0000  63   
                1         29.815354          29.500   28.5  0.1667  76   
male            0         31.516412          31.000   29.0  0.3333  74   
                1         26.977778          27.750   27.0  0.4167  80   
Total Survivors           29.881135          30.000   28.0  0.1667  80   

                                std  
                                age  
sex             survived             
female          0         13.476880  
                1         14.769278  
male            0         13.796345  
                1         15.553884  
Total Survivors           14.406608

This table shows that female female survivors had a higher age than females that died, whereas the inverse is true with males.

In [19]:
pd.pivot_table(data, index= ['pclass', 'sex'], values= "fare", aggfunc= [np.mean, remove_outliers, np.median, min, max, np.std], fill_value= 0,
              margins = True, margins_name= 'Total Survivors')

mean remove_outliers    median    min       max  \
                              fare            fare      fare   fare      fare   
pclass          sex                                                             
1               female  109.412385       95.680738  80.92915  25.70  512.3292   
                male     69.888385       53.460400  49.50420   0.00  512.3292   
2               female   23.234827       19.625000  23.00000  10.50   65.0000   
                male     19.904946       19.175000  13.00000   0.00   73.5000   
3               female   15.324250       12.812500  10.48960   6.75   69.5500   
                male     12.415462        8.668225   7.89580   0.00   69.5500   
Total Survivors          33.295479       19.585400  14.45420   0.00  512.3292   

                              std  
                             fare  
pclass          sex                
1               female  82.885854  
                male    74.079427  
2               female  11.239817  
                male    14.775149  
3               female  11.786512  
                male    11.261638  
Total Survivors         51.738879

This seems to indicate that on average women paid higher fares than men, especially in first class where the average fare for women was about $40 higher.

In [32]:
pd.pivot_table(data, index= 'pclass', columns= "embarked", values='survived', aggfunc='count', fill_value= 0,
              margins = True, margins_name= 'Total Passengers')

embarked,C,Q,S,Total Passengers
pclass,,,,
1,141,3,177,321
2,28,7,242,277
3,101,113,495,709
Total Passengers,270,123,914,1307


This seems to show that most of the passengers embarked at S, that very few non 3rd class passengers embarked at Q, and that only 1 in 7 3rd class passengers came on at C.

In [35]:
pd.pivot_table(data, index= 'pclass', columns= "embarked", values='survived', aggfunc='sum', fill_value= 0,
              margins = True, margins_name= 'Total Survivors')

embarked,C,Q,S,Total Survivors
pclass,,,,
1,97,2,99,198
2,16,2,101,119
3,37,40,104,181
Total Survivors,150,44,304,498


It seems that a little over half of the passengers who embarked at C survived compared to only about 1/3 at the other embarkment points.
This is probably because C was more popular with 1st class passengers than the the other classes.